In [3]:
# Dans le dossier d'analyse, faire un dossier queries et y placer les sequences proteiques des genes
# de virulence que l'on souhaite tester.
# NB. Les sequences peuvent etre des fichiers multi-fasta (plusieurs versions d'un meme gene)
# ou encore des fichiers fasta avec une seule séquence
# Par exemple, le fichier eae.fasta.fasta contient différentes versions de gènes d'intimins (beta, epsilon, etc)
# bfp.fasta contient les genes bfp tires d'un plasmide
# ehxA contient la sequence de reference pour l'hemolysine
# nommer correctement ces genes, car ils seront utilises dans la sortie du programme

In [32]:
import os
import glob
import re
from pathlib import Path
from Bio.Seq import Seq
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastpCommandline
#from Bio.Alphabet import IUPAC

In [33]:
def get_files(start_dir, file_type):
    """ Given a directory, make a list of files
    This function will find all files recursively.
    """
    files = []
    for filename in Path(start_dir).glob('**/*.' + file_type):
        files.append(str(filename))
    return files

In [34]:
# Preparation of a database of CDS from all strains!
cwd = os.getcwd()
if not os.path.exists(cwd + '/database'):
    os.mkdir(cwd + '/database')
if not os.path.exists(cwd + '/xml'):
    os.mkdir(cwd + '/xml')
    
gbk_files = get_files("./MS_minION_all_gbk", "gbk")

# Doing it on a bunch of gbk files
for gbk_i, gbk_file in enumerate(gbk_files):
    
    # Work on UNIX filesystems, name the fasta file from the gbk file
    outfile_name = os.path.basename(gbk_file).replace('.gbk', '')
    print('Adding ', outfile_name, sep=' ')
    
    # Prepare a fasta file for writing
    with open("database/" + outfile_name + '.fasta', "w") as output_handle:
        records = SeqIO.parse(gbk_file, "genbank")

        for record in records:
            for i_feature, feature in enumerate(record.features):
            
                if feature.type == 'CDS':
                    if 'gene' in feature.qualifiers: # if a gene tag is present use it the name
                        feature_id = feature.qualifiers['gene'][0] + '_' + record.name +  '_' + outfile_name
                    else:
                        feature_id = feature.qualifiers['locus_tag'][0] + '_' + record.name + '_' + outfile_name
                
                    # Create the sequence object
                    my_seq = Seq(feature.qualifiers['translation'][0],IUPAC.protein)
                
                    # Create a SeqRecord object with the protein sequence
                    simple_seq_r = SeqRecord(my_seq, id=feature_id, description = '')
                    SeqIO.write(simple_seq_r, output_handle, "fasta")

### In a terminal prepare a NCBI blast+ db


In [1]:
### cat *.fasta > MS_strains_v2
### makeblastdb -in t6ss.fasta -input_type fasta  -dbtype prot -title T6SS_stuff -out T6SS

In [38]:
# Perform blastp searches against the MS_strains database
# The MS_strains NCBI+ database contains protein sequences from the genomes of the 244 strains included in the manuscript 
# Output will be in XML folders
# Create xml folder if it not exists
cwd = os.getcwd()
if not os.path.exists(cwd + '/xml'):
    os.mkdir(cwd + '/xml')

# First grab the query files in the queries folder
virulence_factors = get_files ("./queries/virulence_genes", "fasta")

# Next lauch blastp searches from biopython
for f in virulence_factors:
    outname =  os.path.basename(f).replace('.fasta','')
    blastp_cline = NcbiblastpCommandline(query = f, db="MS_strains_v2", evalue=1e-6, outfmt=5, out='xml/' + outname + '.xml')
    stdout, stderr = blastp_cline()



In [41]:
# This code provide nice output of very high-quality aligments
# And give the names of the strains that contains the query sequence
# Will work only if the query contains multiples versions of the same gene, e.g. many slightly related intimins
# Fragile code : The code search for a pattern in alignment titles
# Because our strains begins by Res13 or R13
# Adjust that pattern to reflect your data


# First grab the bastp output files in the xml folder
xml_files = get_files ("./xml", "xml")

p = re.compile('.+\_(R[es]*13[^_]+)\_(\S+)')  # a pattern for identifying our strains in alignemnt titles
E_VALUE_THRESH = 1e-80
S = set()
for f in xml_files:
    virulence_gene_name = os.path.basename(f).replace('.xml','')
    print ("Analyzing", virulence_gene_name, sep=' ')
    result_handle = open(f)
    from Bio.Blast import NCBIXML
    blast_records = NCBIXML.parse(result_handle)
    for blast_record_i, blast_record in enumerate(blast_records, start=1):
        print('* Query #' + repr(blast_record_i) + ' is ' + blast_record.query + '*', end='')
        print('(length=' + repr(blast_record.query_letters) + ') ', end='\n')
        
        for alignment in blast_record.alignments:
            for hsp in alignment.hsps:
                # restrict to near full_length alignment
                if ((hsp.expect < E_VALUE_THRESH) & (hsp.align_length/blast_record.query_letters > 0.95)):
                    #print("Hit found in", f)
                    print("****Alignment****")
                    print("sequence:", alignment.title)
                    m = p.match(alignment.title) # look if the pattern can be found in the alignment title
                    if m:
                        S.add( (virulence_gene_name, m.group(1), m.group(2)) ) # add a tuple to our set S
                    print("length:", alignment.length)
                    print("e value:", hsp.expect)
                    print(hsp.query[0:75] + "...")
                    print(hsp.match[0:75] + "...")
                    print(hsp.sbjct[0:75] + "...")

Analyzing hlyA
* Query #1 is sp|P09545|HLYA_VIBCH Hemolysin OS=Vibrio cholerae serotype O1 (strain ATCC 39315 / El Tor Inaba N16961) OX=243277 GN=hlyA PE=1 SV=2*(length=741) 
* Query #2 is sp|P08715|HLYAP_ECOLX Hemolysin, plasmid OS=Escherichia coli OX=562 GN=hlyA PE=1 SV=1*(length=1024) 
****Alignment****
sequence: gnl|BL_ORD_ID|260641 hlyA_chr_tig17_R13-CV2-pWea-02_chromosome
length: 1024
e value: 0.0
MTTITTAQIKSTLQSAKQSAANKLHSAGQSTKDALKKAAEQTRNAGNRLILLIPKDYKGQGSSLNDLVRTADELG...
MTTITTAQIKSTLQSAKQSAANKLHSAGQSTKDALKKAAEQTRNAGNRLILLIPKDYKGQGSSLNDLVRTADELG...
MTTITTAQIKSTLQSAKQSAANKLHSAGQSTKDALKKAAEQTRNAGNRLILLIPKDYKGQGSSLNDLVRTADELG...
****Alignment****
sequence: gnl|BL_ORD_ID|76610 hlyA_chr_tig18_R13-AF21-pWea-03_chromosome
length: 1024
e value: 0.0
MTTITTAQIKSTLQSAKQSAANKLHSAGQSTKDALKKAAEQTRNAGNRLILLIPKDYKGQGSSLNDLVRTADELG...
MTTITTAQIKSTLQSAKQSAANKLHSAGQSTKDALKKAAEQTRNAGNRLILLIPKDYKGQGSSLNDLVRTADELG...
MTTITTAQIKSTLQSAKQSAANKLHSAGQSTKDALKKAAEQTRNAGNRLILLIPKDYKGQGSSLNDLVRTADELG...
**

* Query #9 is bfpP*(length=249) 
* Query #10 is bfpH*(length=148) 
* Query #11 is bfpI*(length=181) 
* Query #12 is bfpJ*(length=183) 
* Query #13 is bfpK*(length=164) 
* Query #14 is bfpL*(length=149) 
* Query #15 is bfpM*(length=109) 
* Query #16 is bfpT*(length=274) 
* Query #17 is bfpV*(length=129) 
* Query #18 is bfpW*(length=89) 
Analyzing VT2_B
* Query #1 is sp|P09386|STXB_BP933 Shiga-like toxin 2 subunit B OS=Escherichia phage 933W OX=10730 GN=stxB2 PE=1 SV=1*(length=89) 
Analyzing aggR
* Query #1 is sp|P43464|AGGR_ECOLX Transcriptional activator AggR OS=Escherichia coli OX=562 GN=aggR PE=4 SV=1*(length=265) 
Analyzing eae
* Query #1 is KT591191.1_1 Escherichia coli strain EP016_beta1 intimin (eae) gene, complete cds*(length=940) 
****Alignment****
sequence: gnl|BL_ORD_ID|1348419 eae_pnovel_0_tig35_Res13-Sevr-PEB04-11_plasmid_novel_0
length: 935
e value: 0.0
MITHGFYARTRHKHKLKKTFIMLSAGLGLFFYVNQNSFANGENYFKLSSGSKLLTQNAAQDRLFYTLKTGETVAN...
MITHGFYARTRHKHKLKKTFIMLSAGLGLFFYVNQNSFANGE

* Query #3 is KT591262.1_1 Escherichia coli strain EP064_lambda intimin (eae) gene, complete cds*(length=939) 
****Alignment****
sequence: gnl|BL_ORD_ID|1330723 eae_1_chr_tig47_Res13-Sevr-PEA27-14_chromosome
length: 934
e value: 0.0
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
****Alignment****
sequence: gnl|BL_ORD_ID|1327150 eae_1_chr_tig102_Res13-Sevr-PEA24-03_chromosome
length: 934
e value: 0.0
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
****Alignment****
sequence: gnl|BL_ORD_ID|1321686 eae_1_chr_tig88_Res13-Sevr-PEA20-33_chromosome
length: 934
e value: 0.0
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLG

length: 934
e value: 0.0
MITHGFYARTRHKHKLKKTFIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHNSYQNRLFYTLKTGETVAD...
MITHG Y RTRHKHKLKKT IMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTH+SYQNRLFYTLKTGETVAD...
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
****Alignment****
sequence: gnl|BL_ORD_ID|410705 eae_2_chr_tig98_Res13-Croi-PEA15-12_chromosome
length: 934
e value: 0.0
MITHGFYARTRHKHKLKKTFIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHNSYQNRLFYTLKTGETVAD...
MITHG Y RTRHKHKLKKT IMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTH+SYQNRLFYTLKTGETVAD...
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
****Alignment****
sequence: gnl|BL_ORD_ID|115561 eae_1_chr_tig141_R13-AF22-pSow-01_chromosome
length: 934
e value: 0.0
MITHGFYARTRHKHKLKKTFIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHNSYQNRLFYTLKTGETVAD...
MITHG Y RTRHKHKLKKT IMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTH+SYQNRLFYTLKTGETVAD...
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
****Alignment****
sequen

* Query #9 is KT591313.1_1 Escherichia coli strain EP088_mu intimin (eae) gene, complete cds*(length=936) 
****Alignment****
sequence: gnl|BL_ORD_ID|1330723 eae_1_chr_tig47_Res13-Sevr-PEA27-14_chromosome
length: 934
e value: 0.0
MITHGFYARTRHKHKLKKTFIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHNSYQNRLFYTLKTGETVAD...
MITHG Y RTRHKHKLKKT IMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTH+SYQNRLFYTLKTGETVAD...
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
****Alignment****
sequence: gnl|BL_ORD_ID|1327150 eae_1_chr_tig102_Res13-Sevr-PEA24-03_chromosome
length: 934
e value: 0.0
MITHGFYARTRHKHKLKKTFIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHNSYQNRLFYTLKTGETVAD...
MITHG Y RTRHKHKLKKT IMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTH+SYQNRLFYTLKTGETVAD...
MITHGCYTRTRHKHKLKKTLIMLSAGLGLFFYVNQNSFANGENYFKLGSDSKLLTHDSYQNRLFYTLKTGETVAD...
****Alignment****
sequence: gnl|BL_ORD_ID|1321686 eae_1_chr_tig88_Res13-Sevr-PEA20-33_chromosome
length: 934
e value: 0.0
MITHGFYARTRHKHKLKKTFIMLSAGLGLFFYVNQNSFANGENYFKLGSDSK

In [42]:
# Now discover which strains have the complete query genes
L = list(S) # Change our Set to a mutable List
L.sort(key=lambda tup: tup[1]) # Sort by strain
L

[('hlyA', 'R13-AF11-pWea-02', 'chromosome'),
 ('hlyA', 'R13-AF11-pWea-04', 'chromosome'),
 ('hlyA', 'R13-AF11-pWea-05', 'chromosome'),
 ('hlyA', 'R13-AF21-pWea-03', 'chromosome'),
 ('paa', 'R13-AF21-pWea-03', 'plasmid_529'),
 ('hlyA', 'R13-AF22-pSow-01', 'plasmid_1561'),
 ('eae', 'R13-AF22-pSow-01', 'chromosome'),
 ('VT2_A', 'R13-AF31-pSow-01', 'chromosome'),
 ('VT1_A', 'R13-AF31-pSow-01', 'chromosome'),
 ('hlyA', 'R13-AF31-pSow-02', 'plasmid_novel_0'),
 ('VT1_A', 'R13-AF31-pSow-04', 'chromosome'),
 ('VT2_A', 'R13-AF31-pSow-04', 'chromosome'),
 ('hlyA', 'R13-AF33-pWea-01', 'chromosome'),
 ('hlyA', 'R13-AF33-pWea-02', 'chromosome'),
 ('hlyA', 'R13-AF33-pWea-03', 'chromosome'),
 ('hlyA', 'R13-AF33-pWea-04', 'chromosome'),
 ('hlyA', 'R13-AF33-pWea-05', 'chromosome'),
 ('paa', 'R13-CV2-pWea-02', 'chromosome'),
 ('hlyA', 'R13-CV2-pWea-02', 'chromosome'),
 ('eae', 'Res13-Croi-PEA15-12', 'chromosome'),
 ('hlyA', 'Res13-Croi-PEA15-12', 'plasmid_1561'),
 ('paa', 'Res13-Croi-PEA15-12', 'chromoso

### End of analysis

### Extra pieces of code

#### Code below explain how the bfp query was created

In [8]:
'''Now we are looking for bundle forming pili genes (bfp) in our strains
We will use the Genbank Accession number AB024946 (S.abraham et al. )
Go get the bfp gene in this Genbank record of this plasmid
and then prepare a multifasta file with bfp genes present on a plasmid
'''
from Bio import Entrez
Entrez.email ="jean-simon.brouard@canada.ca"
handle = Entrez.efetch(db="nucleotide", id='AB024946', rettype="gb", retmode="text")
with open('./AB024946.gb', "w") as fp:
    fp.write(handle.read())
    
# Read only 1 record
record = SeqIO.read("AB024946.gb", "genbank")

# Prepare a fasta file for writing
with open('queries/bfp.fasta', "w") as output_handle:
        
    for i_feature, feature in enumerate(record.features):
        if feature.type == 'CDS':
            if 'gene' in feature.qualifiers: # if a gene tag is present use it the name
                if 'bfp' in feature.qualifiers['gene'][0]:
                    feature_id = feature.qualifiers['gene'][0]
                    # Create the sequence object
                    my_seq = Seq(feature.qualifiers['translation'][0],IUPAC.protein)
                
                    # Create a SeqRecord object with the protein sequence
                    simple_seq_r = SeqRecord(my_seq, id=feature_id, description = '')
                    SeqIO.write(simple_seq_r, output_handle, "fasta")

#### Just to show how we can iterate on blast record descriptions

In [ ]:
# Code for outputing the description lines of blast output
# We have aligments, bit scores and E-value
# Note that there is 1 BLAST record object/query if many sequences are given as query
# In addition, hits with good E-value are found in all strains, but one will want to look
# at the aligments to determine if the eae gene is present or not (see the cell below)!
E_VALUE_THRESH = 1e-80
for f in xml_files:
    print ("Analyzing", f, sep=' ')
    result_handle = open(f)
    from Bio.Blast import NCBIXML
    blast_records = NCBIXML.parse(result_handle)
    for blast_record in blast_records:
        for description in blast_record.descriptions:
            print(description)
